# Open Market 자동화
## Overview

### 인식: 인간처리에 좀 더 가깝게.

### 개발 리스트

  1. 복수 오픈마켓 주문정보 수집하여 메일로 알림
  2. 복수 오픈마켓 재고정보 수집하여 메일로 알림
  3. 복수 오픈마켓 부가세 신고자료 취합하여 엑셀로 정리
  
### 2개의 version

1. 브라우저 핸들링 (Selenium)
2. 공식 REST API

### Open Market List

- 지마켓 (http://www.gmarket.co.kr)
- 옥션 (http://www.auction.co.kr/
- 11번가 (https://www.11st.co.kr/html/main.html)
- 스마트스토어 (https://sell.smartstore.naver.com)



## Version1. 브라우저 핸들링 (selenium)

## 1-1. 복수 오픈마켓 주문정보 수집하여 메일로 알림

# <공통> (각 사이트마다)

1. 초기 설정 (브라우저 설정, 브라우저 위치 등)
2. 로그인 페이지 접속
3. 로그인 시도
4. 주문정보 확인 사이트로 이동
5. 주문정보 확인 필터링 걸기
6. 엑셀 다운로드 or crawling
7. 데이터 가공(엑셀 합치기 or HTML 만들기)
8. 메일 보내기

#### ESM Plus(Gmarket. & Auction)

##### (0) 로그인 페이지 접속

In [114]:
## 1. 초기 설정 (브라우저 설정, 브라우저 환경설정(header 등) )
"""
정말 많은 brower가 있습니다만,
본 예제에서는 OS에 독립적으로 실행되길 원하므로,
보편적인 Chrome을 사용하겠습니다.
만약 chrome driver가 없다면 아래 링크에서 설치하십시요.
- http://chromedriver.storage.googleapis.com/index.html
"""

from selenium import webdriver


# TODO: Option에 대한 설명 필요.
# options = None
options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    "download.default_directory": '.'
})
driver = webdriver.Chrome(options=options)
driver.maximize_window()
driver.implicitly_wait(30)


### 아래와 같은 많은 브라우저들이 있습니다.

# driver = webdriver.FireFox
# webdriver.FirefoxProfile

# driver = webdriver.Ie
# driver = webdriver.Opera

# driver = webdriver.PhantomJS

# driver = webdriver.Remote
# webdriver.DesiredCapabilities

# driver = webdriver.safairi

## ...



#### ESM Plus(Gmarket. & Auction)
##### (2) 로그인 페이지 접속

In [115]:
# 관리자 로그인 페이지 접속
login_url = "https://www.esmplus.com/Member/SignIn/LogOn"

driver.get(login_url) # onload wait 포함.


##### (3) 로그인 시도

In [116]:
esm_userid = 'market25'
esm_password = 'hsh981129*'

# id_input 찾기.
id_input = driver.find_element_by_xpath('//*[@id="Id"]')
# password_input 찾기.
password_input = driver.find_element_by_xpath('//*[@id="Password"]')
# login_button 찾기.
login_button = driver.find_element_by_xpath('//*[@id="btnLogOn"]/img')

actions = webdriver.ActionChains(driver)

actions.move_to_element(id_input).click().send_keys(esm_userid)
actions.move_to_element(password_input).click().send_keys(esm_password)
actions.move_to_element(login_button).click()
actions.perform()

actions.reset_actions()


#### 주문정보 확인 사이트로 이동

In [117]:
# is redirected to admin/home
# print(driver.current_url)
# print(driver.current_window_handle)
# print(driver.desired_capabilities)

# TODO: Driver.switch_to 객체에 대해 공부 필요..(문서에 없음.)

actions = webdriver.ActionChains(driver)

order_manage_button = driver.find_element_by_xpath('//*[@id="TDM002"]/a')
actions.move_to_element(order_manage_button).click()

new_order_button = driver.find_element_by_xpath('//*[@id="TDM105"]/a')
actions.move_to_element(new_order_button).click()

actions.perform()
actions.reset_actions()




#### 주문정보 확인 필터링 걸기

In [118]:
### 우리는 최근 한달 신규 주문만 조회하겠습니다.

actions = webdriver.ActionChains(driver)

# 사실 피료 없다.
# order_tab = driver.find_element_by_xpath('//*[@id="tTDM105"]/a[2]')
# actions.move_to_element(order_tab).click()
# actions.perform()
# actions.reset_actions()

driver.switch_to.window(driver.window_handles[0])
driver.switch_to.default_content()
driver.switch_to.frame('ifm_TDM105')


month1_filter = driver.find_element_by_xpath('//*[@id="divSearch"]/div/div[1]/table/tbody/tr[1]/td[1]/span[4]/a')
actions.move_to_element(month1_filter).click()

actions.perform()
actions.reset_actions()

#### 엑셀 다운로드 or crawling

In [120]:
download_excel = driver.find_element_by_xpath('//*[@id="excelDown"]')
actions.move_to_element(download_excel).click()

actions.perform()
actions.reset_actions()

# https://selenium-python.readthedocs.io/api.html#module-selenium.webdriver.common.alert

alerts = driver.switch_to.alert
alerts.accept()


print(driver.window_handles)
print(dir(driver.window_handles[1]))
print(driver.window_handles[1])

parent_handle = driver.current_window_handle
for win_handle in driver.window_handles:
    driver.switch_to.window(win_handle)
    if driver.title == '엑셀분할다운로드설정':
        actions.move_to_element(driver.find_element_by_xpath('//*[@id="popContents"]/div/a[1]/img')).click()
        actions.perform()
        actions.reset_actions()
        driver.close()
        break

        driver.switch_to.window(parent_handle)
        

# driver.switch_to.window(driver.window_handles[-1])



NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615355 (d5698f682d8b2742017df6c81e0bd8e6a3063189),platform=Mac OS X 10.13.5 x86_64)


#### 데이터 가공(엑셀 합치기 or HTML 만들기)
#### 메일 보내기





## 2. 복수 오픈마켓 재고정보 수집하여 메일로 알림

0. 각 OpenMarket 점검 시간인지 checking
1. 로그인 페이지 접속
2. 로그인 시도
3. 재고 정보 확인 사이트로 이동
4. 재고 정보 수집 필터링 걸기
5. 엑셀 다운로드 or crawling
6. 데이터 가공(엑셀 합치기 or HTML 만들기)
7. 메일 보내기

## 3. 복수 오픈마켓 부가세 신고자료 취합하여 엑셀로 정리

0. 각 OpenMarket 점검 시간인지 checking
1. 로그인 페이지 접속
2. 로그인 시도
3. 부가세 확인 사이트로 이동
4. 부가세 정보 수집 필터링 걸기
5. 엑셀 다운로드 or crawling
6. 데이터 가공(엑셀 합치기 or HTML 만들기)
7. 메일 보내기

#### REST API 사용 버전

**(보류)**